## MAGNet - PVCA 

R4.0.5

In [11]:
library(dplyr)
library(stringr)
library("SummarizedExperiment")

options(repr.plot.width=16, repr.plot.height=10)

### Download data to preferred directory

```
#!/bin/sh
wget --output-document data.zip 'https://data.dieterichlab.org/s/3gCTLGT4DaAqaqb/download' --header 'Accept: text/html' --header 'Connection: keep-alive'
unzip -o data.zip
rm data.zip
rm MAGNetApp/magnetique.sqlite MAGNetApp/data/colData.txt
rm -r MAGNetApp/data/DGE MAGNetApp/data/DTU MAGNetApp/data/networks MAGNetApp/data/RBP
```

In [4]:
dirloc <- '/prj/MAGE/analysis/genetonic/PVCA'

load(file.path(dirloc, 'MAGNetApp', 'data', 'MAGNet.RData'))

colData$AFib <- factor(colData$AFib, levels=c("No", "Yes"))
colData$VTVF <- factor(colData$VTVF, levels=c("No", "Yes"))
colData$Diabetes <- factor(colData$Diabetes, levels=c("No", "Yes"))
colData$Hypertension <- factor(colData$VTVF, levels=c("No", "Yes"))
colData$LibraryPool <- factor(colData$LibraryPool)

In [8]:
head(colData)
all.equal(colnames(countData), rownames(colData))
dim(countData)

,Run,Experiment,LibraryPool,TIN,RIN,DuplicationRate,TissueSource,Etiology,Race,Age,Sex,Weight,Height,HW,LVMass,AFib,VTVF,Diabetes,Hypertension,LVEF
,<chr>,<chr>,<fct>,<dbl>,<dbl>,<dbl>,<chr>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<dbl>
C00039,SRR10676821,SRX7354049,Magnet_10,72.76478,8.4,0.321948,NF,NFD,AA,18,M,70,175,NA,NA,No,No,No,No,0.37
C00055,SRR10676822,SRX7354050,Magnet_11,74.29391,9.1,0.461090,NF,NFD,AA,26,M,85,183,NA,NA,No,No,No,No,NA
C00074,SRR10676823,SRX7354051,Magnet_09,77.44517,7.8,0.338003,NF,NFD,AA,17,M,68,173,400,NA,No,No,No,No,0.27
C00085,SRR10676824,SRX7354319,Magnet_06,69.51659,9.4,0.420441,NF,NFD,AA,59,F,66,157,380,NA,No,No,No,No,0.60
C00105,SRR10676825,SRX7354320,Magnet_03,73.60743,8.8,0.290037,NF,NFD,AA,59,F,87,157,NA,NA,No,No,No,No,0.55
C00120,SRR10676826,SRX7354321,Magnet_12,30.35077,8.6,0.595905,NF,NFD,C,50,M,87,175,640,NA,No,No,No,No,0.65


[1] TRUE

[1] 41886   354

### We try to see how the inferred surrogate variables correlate with the covariates that we did not include in the model...

In [10]:
library(sva)

In [12]:
findDeseqFactorsSingle <- function(count_data)
{
  loggeomeans <- rowMeans(log(count_data))
  deseqFactors <- apply(count_data, 2, deseq, loggeomeans = loggeomeans) 
  deseqFactors
}

deseq <- function(x, loggeomeans) {
  finitePositive <- is.finite(loggeomeans) & x > 0
  if (any(finitePositive)) {
    res <- exp(median((log(x) - loggeomeans)[finitePositive], na.rm = TRUE))
  } else {
    print(utils::head(x))
    stop("Can't normalise accross a condition.
         Too many zero expressed genes. ")
  }
  res
}

sizeFactors <- findDeseqFactorsSingle(countData)
normData <- t(t(countData) / findDeseqFactorsSingle(countData))


In [13]:
# full model
# including both the adjustment variables and the variable of interest (here we assume none)
# ~Etiology+LibraryPool+TIN+RIN+DuplicationRate+Race+Age+Sex+Weight+Height+HW+AFib+VTVF+Diabetes+Hypertension+LVEF
mod = model.matrix(~Etiology, data=colData)

# null model
# only the adjustment variables
mod0 <- model.matrix(~1,data=colData)

In [14]:
# estimate number of latent factors
# n.sv = num.sv(normData, mod, method="leek")

# skip and fix for now to be the number of real adjustment variables (drop LVMass)
n.sv <- 5

# fix number of surrogate variables...
svseq <- svaseq(normData, mod, mod0, n.sv = n.sv)

# sv is a matrix whose columns correspond to the estimated surrogate variables (can be used in downstream analyses)
# pprob.gam is the posterior probability that each gene is associated with one or more latent variables
# pprob.b is the posterior probability that each gene is associated with the variables of interest

Number of significant surrogate variables is:  5 
Iteration (out of 5 ):1  2  3  4  5  

In [ ]:
# Because we actually do know the real variables (assuming there are no other sources of variation), 
# we can see how well the SVA method did at recovering these variables 

In [16]:
for (n in c('LibraryPool', 'TIN', 'RIN', 'DuplicationRate', 'Race', 'Age', 'Sex', 'Weight', 'Height', 'HW', 'AFib', 'VTVF', 'Diabetes', 'Hypertension', 'LVEF')){
   png(file.path(dirloc, paste(n, 'SVA5.png', sep='-')), width=600, height=350)
    par(mfrow = c(2, 3), mar = c(3,5,3,1))
    for (i in 1:n.sv) {
      stripchart(svseq$sv[, i] ~ colData[[n]], vertical = TRUE, main = paste0("SV", i))
      abline(h = 0)
     }
   dev.off()
}

### PVCA

In [18]:
# based on https://bioconductor.statistik.tu-dortmund.de/packages/3.7/workflows/vignettes/ExpressionNormalizationWorkflow

library(Biobase)
library(limma)
library(lme4)
library(matrixStats)
library(pvca)
library(vsn)

expSetobj <- function(expression, covariates, cov_desc=NULL)
{
    expression <- as.matrix(expression)
    if(is.null(cov_desc)) {
        cov_desc <- colnames(covariates)
    }
    metadata <- data.frame(labelDescription=cov_desc, 
                           row.names=colnames(covariates))
    phenoData <- new("AnnotatedDataFrame", data=covariates,
                     varMetadata=metadata)
    exp_datObj <- ExpressionSet(assayData=expression, phenoData=phenoData)
    return(exp_datObj)
}


## PVCA by Pierre R.Bushel and Jianying Li
pvcaBatchAssess <- function (abatch, batch_factors, threshold)
{
    theDataMatrix <- exprs(vsn2(abatch, verbose=FALSE))
    dataRowN <- nrow(theDataMatrix)
    dataColN <- ncol(theDataMatrix)
    theDataMatrixCentered <- matrix(data=0, nrow=dataRowN,
                                    ncol=dataColN)
    theDataMatrixCentered_transposed <- apply(theDataMatrix, 1,
                                             scale, center=TRUE, scale=FALSE)
    theDataMatrixCentered <- t(theDataMatrixCentered_transposed)
    theDataCor <- cor(theDataMatrixCentered)
    eigenData <- eigen(theDataCor)
    eigenValues <- eigenData$values
    ev_n <- length(eigenValues)
    eigenVectorsMatrix <- eigenData$vectors
    eigenValuesSum <- sum(eigenValues)
    percents_PCs <- eigenValues / eigenValuesSum
    expInfo <- pData(abatch)[, batch_factors]
    exp_design <- as.data.frame(expInfo)
    expDesignRowN <- nrow(exp_design)
    expDesignColN <- ncol(exp_design)
    my_counter_2 <- 0
    my_sum_2 <- 1
    for (i in ev_n:1) {
        my_sum_2 <- my_sum_2 - percents_PCs[i]
        if ( (my_sum_2) <= threshold) {
            my_counter_2 <- my_counter_2 + 1
        }
    }
    if (my_counter_2 < 3) {
        pc_n <- 3
    }
    else {
        pc_n <- my_counter_2
    }
    pc_data_matrix <- matrix(data=0, nrow= (expDesignRowN *
                                                   pc_n), ncol=1)
    mycounter <- 0
    for (i in 1:pc_n) {
        for (j in 1:expDesignRowN) {
            mycounter <- mycounter + 1
            pc_data_matrix[mycounter, 1] <- eigenVectorsMatrix[j,i]
        }
    }
    AAA <- exp_design[rep(1:expDesignRowN, pc_n), ]
    Data <- cbind(AAA, pc_data_matrix)
    variables <- c(colnames(exp_design))
    for (i in 1:length(variables)) {
        Data$variables[i] <- as.factor(Data$variables[i])
    }
    op <- options(warn= (-1))
    effects_n <- expDesignColN + 1  
    randomEffectsMatrix <- matrix(data=0, nrow=pc_n, ncol=effects_n)
    model.func <- c()
    index <- 1
    for (i in 1:length(variables)) {
        mod <- paste("(1|", variables[i], ")", sep="")
        model.func[index] <- mod
        index <- index + 1
    }
    function.mods <- paste(model.func, collapse=" + ")
    for (i in 1:pc_n) {
        y <- (((i - 1) * expDesignRowN) + 1)
        funct <- paste("pc_data_matrix", function.mods, sep=" ~ ")
        Rm1ML <- lmer(funct, Data[y: ( ( (i - 1) * expDesignRowN) +
                     expDesignRowN), ], REML=TRUE, verbose=FALSE,
                            na.action=na.omit)
        randomEffects <- Rm1ML
        randomEffectsMatrix[i, ] <- c(unlist(VarCorr(Rm1ML)),
                                      resid=sigma(Rm1ML)^2)
    }
    effectsNames <- c(names(getME(Rm1ML, "cnms")), "resid")
    randomEffectsMatrixStdze <- matrix(data=0, nrow=pc_n,
                                       ncol=effects_n)
    for (i in 1:pc_n) {
        mySum <- sum(randomEffectsMatrix[i, ])
        for (j in 1:effects_n) {
            randomEffectsMatrixStdze[i, j] <- randomEffectsMatrix[i,
                                                                 j] / mySum
        }
    }
    randomEffectsMatrixWtProp <- matrix(data=0, nrow=pc_n,
                                        ncol=effects_n)
    for (i in 1:pc_n) {
        weight <- eigenValues[i] / eigenValuesSum
        for (j in 1:effects_n) {
            randomEffectsMatrixWtProp[i, j] <- randomEffectsMatrixStdze[i,j] * weight
        }
    }
    randomEffectsSums <- matrix(data=0, nrow=1, ncol=effects_n)
    randomEffectsSums <- colSums(randomEffectsMatrixWtProp)
    totalSum <- sum(randomEffectsSums)
    randomEffectsMatrixWtAveProp <- matrix(data=0, nrow=1,
                                           ncol=effects_n)
    for (j in 1:effects_n) {
        randomEffectsMatrixWtAveProp[j] <- randomEffectsSums[j] / totalSum
    }
    return(list(dat=randomEffectsMatrixWtAveProp, label=effectsNames))
}

snmAnaly <- function(exprs, cov, bv, av, iv, adj=TRUE)
{
    kar_rawdata <- exprs
    kar_bio.var <- as.data.frame(cov[,bv])
    kar_adj.var <- as.data.frame(cov[,av])
    rownames(kar_adj.var) <- rownames(cov)
    colnames(kar_adj.var) <- av
    kar_int.var <- as.data.frame(cov[,iv])
    colnames(kar_int.var) <- iv
    int.var <- kar_int.var
    int.var$Array <- as.factor(int.var$Array)
    adj.var <- model.matrix(~., kar_adj.var)
    bio.var <- model.matrix(~., kar_bio.var)
    raw.data <- as.matrix(kar_rawdata)
    snm.obj <- snm(raw.data, bio.var, adj.var, int.var, rm.adj=adj, num.iter=5)
    write.table(snm.obj$norm.dat, file="snm_normalized_data.csv", 
                sep=",", col.names=NA)
    return(snm.obj)
}


conTocat <- function(covariates, var_names)
{
    var <- covariates[,var_names]
    len <- dim(var)[2]
    for (i in 1:len) {
        temp <- as.numeric(with(var, cut(var[,i], breaks=
                          quantile(var[,i], probs=seq(0, 1, by=0.25)), 
                                                  include.lowest=TRUE)))
        covariates <- cbind(covariates, temp)
        colnames(covariates)[ncol(covariates)] <- 
                           paste(var_names[i], "_cat", sep="")
    }
    return(covariates)
}

PVCA (Bushel, 2013) estimates the proportion of the variance of the principal components of the gene expression data that can be attributed to each of the given covariates.The remaining fraction is assigned as “residual” variance.It efficiently combines principal component analysis (PCA) to reduce the feature space and variance component analysis (VCA) which fits a mixed linear model using factors of interest as random effects to estimate and partition the total variability. The variance explained is computed as a weighted average of the contributions of each factor to each PC, and you have the option of specifying how many principal components to include, by setting a threshold amount of the variance that needs to be explained by the identified PCs.

In [20]:
# we first need to categorize the continuous covariates

colData$TIN <- cut(colData$TIN, 
                   breaks = quantile(colData$TIN, by=0.25, na.rm = TRUE), 
                   include.lowest = TRUE)
colData$RIN <- cut(colData$RIN, 
                   breaks = quantile(colData$RIN, by=0.25, na.rm = TRUE), 
                   include.lowest = TRUE)
colData$DuplicationRate <- cut(colData$DuplicationRate, 
                   breaks = quantile(colData$DuplicationRate, by=0.25, na.rm = TRUE), 
                   include.lowest = TRUE)
colData$Age <- cut(colData$Age, 
                   breaks = quantile(colData$Age, by=0.25, na.rm = TRUE), 
                   include.lowest = TRUE)
colData$Weight <- cut(colData$Weight, 
                   breaks = quantile(colData$Weight, by=0.25, na.rm = TRUE), 
                   include.lowest = TRUE)
colData$Height <- cut(colData$Height, 
                   breaks = quantile(colData$Height, by=0.25, na.rm = TRUE), 
                   include.lowest = TRUE)
colData$HW <- cut(colData$HW, 
                   breaks = quantile(colData$HW, by=0.25, na.rm = TRUE), 
                   include.lowest = TRUE)
colData$LVEF <- cut(colData$LVEF, 
                   breaks = quantile(colData$LVEF, by=0.25, na.rm = TRUE), 
                   include.lowest = TRUE)

In [21]:
# and remove unused variables
colData <- colData[,-c(1,2,7,15)]

In [22]:
head(colData)

,LibraryPool,TIN,RIN,DuplicationRate,Etiology,Race,Age,Sex,Weight,Height,HW,AFib,VTVF,Diabetes,Hypertension,LVEF
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
C00039,Magnet_10,"(70.1,73.3]","(8.1,8.5]","[0.233,0.378]",NFD,AA,"[15,48]",M,"(66.2,79]","(170,178]",NA,No,No,No,No,"(0.2,0.52]"
C00055,Magnet_11,"(73.3,81.5]","(8.9,10]","(0.421,0.517]",NFD,AA,"[15,48]",M,"(79,91]","(178,196]",NA,No,No,No,No,NA
C00074,Magnet_09,"(73.3,81.5]","[5.8,8.1]","[0.233,0.378]",NFD,AA,"[15,48]",M,"(66.2,79]","(170,178]","(372,448]",No,No,No,No,"(0.2,0.52]"
C00085,Magnet_06,"(60.6,70.1]","(8.9,10]","(0.378,0.421]",NFD,AA,"(56,62]",F,"[27,66.2]","[55,163]","(372,448]",No,No,No,No,"(0.52,0.8]"
C00105,Magnet_03,"(73.3,81.5]","(8.5,8.9]","[0.233,0.378]",NFD,AA,"(56,62]",F,"(79,91]","[55,163]",NA,No,No,No,No,"(0.52,0.8]"
C00120,Magnet_12,"[23.4,60.6]","(8.5,8.9]","(0.517,0.935]",NFD,C,"(48,56]",M,"(79,91]","(170,178]","(558,923]",No,No,No,No,"(0.52,0.8]"


In [23]:
# convert to ExpressionSet
# inpData <- expSetobj(countData, colData)

# use norm - uncomment otherwise
sizeFactors <- findDeseqFactorsSingle(countData)
normData <- t(t(countData) / findDeseqFactorsSingle(countData))
inpData <- expSetobj(normData, colData)

In [24]:
# set the covariates whose effect size on the data needs to be calculated
cvrts_eff_var <- colnames(colData)
## PVCA Threshold Value is the percentile value of the minimum amount of the variabilities that the selected principal components need to explain, 
## here requiring 75% of the expression variance to be captured by the PCs
pct_thrsh <- 0.75 
## Perform the PVCA analysis
pvcaObj <- pvcaBatchAssess(inpData, cvrts_eff_var, pct_thrsh)

# note that the model is singular...
# we could try reducing the number of covariates, etc.? -> there is no straighforward solution though...

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSi

In [25]:
# plot

png(file.path(dirloc, 'PVCA_norm.png'), width=800, height=400)
bp <- barplot(pvcaObj$dat,
              ylab=expression(bold("Weighted Average Proportion Variance")),
              ylim=c(0,1.1), col=c("navy"), las=2, font=4,
              main="Principal Variant Component Analysis Estimation")
axis(1,at=bp,labels=pvcaObj$label,cex.axis=1,las=2,font=4)
values <- pvcaObj$dat
new_values <- round(values,3)
text(bp,pvcaObj$dat,labels=new_values,pos=3,cex=1,font=4)
dev.off()


png 
  2

In [26]:
saveRDS(pvcaObj, file.path(dirloc, 'pvcaObj.RDS'))

In [ ]:
# Surrogate Variable Analysis ...

# These are appended as new covariates to the existing list of covariates, and 
# we also add a step to convert them to categorical variables so as to estimate their contributions to the total variance.

In [27]:
# choose a biological variable that is to be used to calculate the surrogate variables
biol_var_sva <- "Etiology" 
# Perform SVA as above
pheno <- pData(inpData)
expData <- exprs(inpData) # use norm
mod  <- model.matrix(~as.factor(pheno[, biol_var_sva]), data=pheno)
mod0 <- model.matrix(~1,data=pheno)

n.sv <- 5
svobj <- svaseq(expData, mod, mod0, n.sv=n.sv)

Number of significant surrogate variables is:  5 
Iteration (out of 5 ):1  2  3  4  5  

In [28]:
# reformat
srg_var <- as.data.frame(svobj$sv)     
colnames(srg_var) <- paste("sv", 1:ncol(srg_var), sep="")
pData(inpData) <- cbind(pData(inpData), srg_var)
varMetadata(inpData)[colnames(srg_var),] <- colnames(srg_var)
svobj$expSetobject <- inpData

In [29]:
## append to the ExpressionSet object
inpData_sv <- svobj$expSetobject

Computing the correlation between the surrogate variables and the covariates...

We want to identify if a given surrogate variable is entirely independent of all the existing covariates or if it correlates with some technical factor, etc.

In [30]:
## Fit a generalized linear model for sv1
glm.sv1 <- glm(pData(inpData_sv)[,"sv1"] ~ pData(inpData_sv)[,"LibraryPool"] + 
               pData(inpData_sv)[,"TIN"] + pData(inpData_sv)[,"RIN"] +
               pData(inpData_sv)[,"DuplicationRate"] + pData(inpData_sv)[,"Etiology"] +
               pData(inpData_sv)[,"Race"] + pData(inpData_sv)[,"Sex"] + pData(inpData_sv)[,"Weight"] +
               pData(inpData_sv)[,"Height"] + pData(inpData_sv)[,"HW"] + pData(inpData_sv)[,"AFib"] +
               pData(inpData_sv)[,"VTVF"] + pData(inpData_sv)[,"Diabetes"] + pData(inpData_sv)[,"Hypertension"] +
               pData(inpData_sv)[,"LVEF"]) 
summary(glm.sv1)


Call:
glm(formula = pData(inpData_sv)[, "sv1"] ~ pData(inpData_sv)[, 
    "LibraryPool"] + pData(inpData_sv)[, "TIN"] + pData(inpData_sv)[, 
    "RIN"] + pData(inpData_sv)[, "DuplicationRate"] + pData(inpData_sv)[, 
    "Etiology"] + pData(inpData_sv)[, "Race"] + pData(inpData_sv)[, 
    "Sex"] + pData(inpData_sv)[, "Weight"] + pData(inpData_sv)[, 
    "Height"] + pData(inpData_sv)[, "HW"] + pData(inpData_sv)[, 
    "AFib"] + pData(inpData_sv)[, "VTVF"] + pData(inpData_sv)[, 
    "Diabetes"] + pData(inpData_sv)[, "Hypertension"] + pData(inpData_sv)[, 
    "LVEF"])

Deviance Residuals: 
      Min         1Q     Median         3Q        Max  
-0.068446  -0.011751  -0.000268   0.010948   0.091576  

Coefficients: (1 not defined because of singularities)
                                                      Estimate Std. Error
(Intercept)                                         -0.0544406  0.0121796
pData(inpData_sv)[, "LibraryPool"]Magnet_02         -0.0213014  0.0081037
pData(inpData_sv

In [31]:
glm.sv2 <- glm(pData(inpData_sv)[,"sv2"] ~ pData(inpData_sv)[,"LibraryPool"] + 
               pData(inpData_sv)[,"TIN"] + pData(inpData_sv)[,"RIN"] +
               pData(inpData_sv)[,"DuplicationRate"] + pData(inpData_sv)[,"Etiology"] +
               pData(inpData_sv)[,"Race"] + pData(inpData_sv)[,"Sex"] + pData(inpData_sv)[,"Weight"] +
               pData(inpData_sv)[,"Height"] + pData(inpData_sv)[,"HW"] + pData(inpData_sv)[,"AFib"] +
               pData(inpData_sv)[,"VTVF"] + pData(inpData_sv)[,"Diabetes"] + pData(inpData_sv)[,"Hypertension"] +
               pData(inpData_sv)[,"LVEF"]) 
summary(glm.sv2)


Call:
glm(formula = pData(inpData_sv)[, "sv2"] ~ pData(inpData_sv)[, 
    "LibraryPool"] + pData(inpData_sv)[, "TIN"] + pData(inpData_sv)[, 
    "RIN"] + pData(inpData_sv)[, "DuplicationRate"] + pData(inpData_sv)[, 
    "Etiology"] + pData(inpData_sv)[, "Race"] + pData(inpData_sv)[, 
    "Sex"] + pData(inpData_sv)[, "Weight"] + pData(inpData_sv)[, 
    "Height"] + pData(inpData_sv)[, "HW"] + pData(inpData_sv)[, 
    "AFib"] + pData(inpData_sv)[, "VTVF"] + pData(inpData_sv)[, 
    "Diabetes"] + pData(inpData_sv)[, "Hypertension"] + pData(inpData_sv)[, 
    "LVEF"])

Deviance Residuals: 
      Min         1Q     Median         3Q        Max  
-0.144087  -0.015522   0.002433   0.015826   0.097397  

Coefficients: (1 not defined because of singularities)
                                                      Estimate Std. Error
(Intercept)                                          0.0749445  0.0174754
pData(inpData_sv)[, "LibraryPool"]Magnet_02         -0.0167360  0.0116272
pData(inpData_sv

In [32]:
glm.sv3 <- glm(pData(inpData_sv)[,"sv3"] ~ pData(inpData_sv)[,"LibraryPool"] + 
               pData(inpData_sv)[,"TIN"] + pData(inpData_sv)[,"RIN"] +
               pData(inpData_sv)[,"DuplicationRate"] + pData(inpData_sv)[,"Etiology"] +
               pData(inpData_sv)[,"Race"] + pData(inpData_sv)[,"Sex"] + pData(inpData_sv)[,"Weight"] +
               pData(inpData_sv)[,"Height"] + pData(inpData_sv)[,"HW"] + pData(inpData_sv)[,"AFib"] +
               pData(inpData_sv)[,"VTVF"] + pData(inpData_sv)[,"Diabetes"] + pData(inpData_sv)[,"Hypertension"] +
               pData(inpData_sv)[,"LVEF"]) 
summary(glm.sv3)


Call:
glm(formula = pData(inpData_sv)[, "sv3"] ~ pData(inpData_sv)[, 
    "LibraryPool"] + pData(inpData_sv)[, "TIN"] + pData(inpData_sv)[, 
    "RIN"] + pData(inpData_sv)[, "DuplicationRate"] + pData(inpData_sv)[, 
    "Etiology"] + pData(inpData_sv)[, "Race"] + pData(inpData_sv)[, 
    "Sex"] + pData(inpData_sv)[, "Weight"] + pData(inpData_sv)[, 
    "Height"] + pData(inpData_sv)[, "HW"] + pData(inpData_sv)[, 
    "AFib"] + pData(inpData_sv)[, "VTVF"] + pData(inpData_sv)[, 
    "Diabetes"] + pData(inpData_sv)[, "Hypertension"] + pData(inpData_sv)[, 
    "LVEF"])

Deviance Residuals: 
      Min         1Q     Median         3Q        Max  
-0.176157  -0.028698  -0.003764   0.022816   0.222160  

Coefficients: (1 not defined because of singularities)
                                                      Estimate Std. Error
(Intercept)                                          0.0012833  0.0273612
pData(inpData_sv)[, "LibraryPool"]Magnet_02         -0.0625124  0.0182047
pData(inpData_sv

In [34]:
glm.sv4 <- glm(pData(inpData_sv)[,"sv4"] ~ pData(inpData_sv)[,"LibraryPool"] + 
               pData(inpData_sv)[,"TIN"] + pData(inpData_sv)[,"RIN"] +
               pData(inpData_sv)[,"DuplicationRate"] + pData(inpData_sv)[,"Etiology"] +
               pData(inpData_sv)[,"Race"] + pData(inpData_sv)[,"Sex"] + pData(inpData_sv)[,"Weight"] +
               pData(inpData_sv)[,"Height"] + pData(inpData_sv)[,"HW"] + pData(inpData_sv)[,"AFib"] +
               pData(inpData_sv)[,"VTVF"] + pData(inpData_sv)[,"Diabetes"] + pData(inpData_sv)[,"Hypertension"] +
               pData(inpData_sv)[,"LVEF"]) 
summary(glm.sv4)


Call:
glm(formula = pData(inpData_sv)[, "sv4"] ~ pData(inpData_sv)[, 
    "LibraryPool"] + pData(inpData_sv)[, "TIN"] + pData(inpData_sv)[, 
    "RIN"] + pData(inpData_sv)[, "DuplicationRate"] + pData(inpData_sv)[, 
    "Etiology"] + pData(inpData_sv)[, "Race"] + pData(inpData_sv)[, 
    "Sex"] + pData(inpData_sv)[, "Weight"] + pData(inpData_sv)[, 
    "Height"] + pData(inpData_sv)[, "HW"] + pData(inpData_sv)[, 
    "AFib"] + pData(inpData_sv)[, "VTVF"] + pData(inpData_sv)[, 
    "Diabetes"] + pData(inpData_sv)[, "Hypertension"] + pData(inpData_sv)[, 
    "LVEF"])

Deviance Residuals: 
      Min         1Q     Median         3Q        Max  
-0.124318  -0.024523  -0.000557   0.021854   0.143812  

Coefficients: (1 not defined because of singularities)
                                                      Estimate Std. Error
(Intercept)                                          0.0345176  0.0230448
pData(inpData_sv)[, "LibraryPool"]Magnet_02          0.0770188  0.0153328
pData(inpData_sv

In [35]:
glm.sv5 <- glm(pData(inpData_sv)[,"sv5"] ~ pData(inpData_sv)[,"LibraryPool"] + 
               pData(inpData_sv)[,"TIN"] + pData(inpData_sv)[,"RIN"] +
               pData(inpData_sv)[,"DuplicationRate"] + pData(inpData_sv)[,"Etiology"] +
               pData(inpData_sv)[,"Race"] + pData(inpData_sv)[,"Sex"] + pData(inpData_sv)[,"Weight"] +
               pData(inpData_sv)[,"Height"] + pData(inpData_sv)[,"HW"] + pData(inpData_sv)[,"AFib"] +
               pData(inpData_sv)[,"VTVF"] + pData(inpData_sv)[,"Diabetes"] + pData(inpData_sv)[,"Hypertension"] +
               pData(inpData_sv)[,"LVEF"]) 
summary(glm.sv5)


Call:
glm(formula = pData(inpData_sv)[, "sv5"] ~ pData(inpData_sv)[, 
    "LibraryPool"] + pData(inpData_sv)[, "TIN"] + pData(inpData_sv)[, 
    "RIN"] + pData(inpData_sv)[, "DuplicationRate"] + pData(inpData_sv)[, 
    "Etiology"] + pData(inpData_sv)[, "Race"] + pData(inpData_sv)[, 
    "Sex"] + pData(inpData_sv)[, "Weight"] + pData(inpData_sv)[, 
    "Height"] + pData(inpData_sv)[, "HW"] + pData(inpData_sv)[, 
    "AFib"] + pData(inpData_sv)[, "VTVF"] + pData(inpData_sv)[, 
    "Diabetes"] + pData(inpData_sv)[, "Hypertension"] + pData(inpData_sv)[, 
    "LVEF"])

Deviance Residuals: 
      Min         1Q     Median         3Q        Max  
-0.283466  -0.023588   0.002147   0.025983   0.140069  

Coefficients: (1 not defined because of singularities)
                                                      Estimate Std. Error
(Intercept)                                          0.0110040  0.0274284
pData(inpData_sv)[, "LibraryPool"]Magnet_02          0.0073946  0.0182494
pData(inpData_sv

In [ ]:
# Principal Variance Component Analysis of the raw data with the surrogate variables included as covariates

In [36]:
# discretize the continuous surrogate variables 
var_names <- c("sv1", "sv2", "sv3", "sv4", "sv5") 
pData(inpData_sv) <- conTocat(pData(inpData_sv), var_names) 
## Include the surrogate variables as covariates in addition to others
cvrts_eff_var <- c(cvrts_eff_var, "sv1_cat", "sv2_cat", "sv3_cat", "sv4_cat", "sv5_cat")
## again set the PVCA Threshold to explain 75% of the expression variation
pct_thrsh <- 0.75 
## Perform PVCA
pvcaObj_sva <- pvcaBatchAssess(inpData_sv, cvrts_eff_var, pct_thrsh)

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSi

In [37]:
png(file.path(dirloc, 'PVCA_sva.png'), width=800, height=400)
bp <- barplot(pvcaObj_sva$dat,
              ylab=expression(bold("Weighted Average Proportion Variance")),
              ylim=c(0,1.1), col=c("navy"), las=2, font=4,
              main="Principal Variant Component Analysis Estimation")
axis(1,at=bp,labels=pvcaObj_sva$label,cex.axis=1,las=2,font=4)
values <- pvcaObj_sva$dat
new_values <- round(values,3)
text(bp,pvcaObj_sva$dat,labels=new_values,pos=3,cex=1,font=4)
dev.off()

png 
  2

In [38]:
# For comparison, remove TIN and LibraryPool to see how much of it the surrogate variables capture
pvcaObj_sva2 <- pvcaBatchAssess(inpData_sv, cvrts_eff_var[-which(cvrts_eff_var %in% c('LibraryPool', 'TIN'))], pct_thrsh)

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSi

In [40]:
png(file.path(dirloc, 'PVCA_sva1-2.png'), width=800, height=400)
bp <- barplot(pvcaObj_sva2$dat,
              ylab=expression(bold("Weighted Average Proportion Variance")),
              ylim=c(0,1.1), col=c("navy"), las=2, font=4,
              main="Principal Variant Component Analysis Estimation")
axis(1,at=bp,labels=pvcaObj_sva2$label,cex.axis=1,las=2,font=4)
values <- pvcaObj_sva2$dat
new_values <- round(values,3)
text(bp,pvcaObj_sva2$dat,labels=new_values,pos=3,cex=1,font=4)
dev.off()

png 
  2

In [41]:
# In summary, SV1 and SV2 capture a lot of the variance that was in LibraryPool and TIN...